# Insurance Fraud Analysis

# Importing Libraries

In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Reading Dataset

In [ ]:
insurance_data = pd.read_csv("../dataset/Insurance Dataset.csv")
insurance_data.head()

# EDA

In [ ]:
insurance_data.info()

In [ ]:
# Keeping a copy of the original data set, we might need later
insurance_data_copy = insurance_data.copy()

In [ ]:
insurance_data["Weight_baby"].unique()

## Feature Engineering: Drop useless column

In [ ]:
cat = ["Area_Service", "Hospital County", "Age", "Gender", "Cultural_group", "ethnicity", "Admission_type", "Home or self care,", "apr_drg_description", "Surg_Description", "Abortion", "Emergency dept_yes/No"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

In [ ]:
for column in cat:
    label_encoder.fit(insurance_data[column])
    insurance_data[column] = label_encoder.transform(insurance_data[column])

In [ ]:
insurance_data["Days_spend_hsptl"].replace(["120 +"], "120", inplace = True)
insurance_data["Days_spend_hsptl"] = pd.to_numeric(insurance_data["Days_spend_hsptl"])
insurance_data["Days_spend_hsptl"].unique()

In [ ]:
insurance_data.info()

In [ ]:
insurance_data.dropna(inplace = True)

In [ ]:
insurance_data["Mortality risk"] = insurance_data["Mortality risk"].astype(int)
insurance_data["Hospital Id"] = insurance_data["Hospital Id"].astype(int)

In [ ]:
insurance_data.info()

In [ ]:
X = insurance_data.drop(columns = ["Result"])
y = insurance_data["Result"]

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25)

In [ ]:
from sklearn.feature_selection import chi2
f_p_values = chi2(Xtrain, ytrain)

In [ ]:
f_values = pd.Series(f_p_values[0])
p_values = pd.Series(f_p_values[1])

In [ ]:
p_values.index = Xtrain.columns
f_values.index = Xtrain.columns

In [ ]:
f_values_d = pd.DataFrame({'Columns': f_values.index, "f_value": f_p_values[0]})


In [ ]:
f_values_d = f_values_d.sort_values(by = "f_value", ascending = False)
f_values_d.reset_index(inplace = True)
f_values_d.drop(columns = ["index"], inplace = True)
f_values_d

In [ ]:
p_values_d = pd.DataFrame({'Columns': p_values.index, "p_value": f_p_values[1]})

In [ ]:
p_values_d = p_values_d.sort_values(by = "p_value", ascending = True)
p_values_d.reset_index(inplace = True)
p_values_d.drop(columns = ["index"], inplace = True)
p_values_d

In [ ]:
p_values_d[p_values_d["p_value"] < .8]

In [ ]:
insurance_data.columns

In [ ]:
imp_columns = ["Hospital County", "Hospital_Id", "Cultural_group", "ethnicity", "Days_spend_hsptl", "Admission_type", "Home or self care,", "ccs_procedure_code", "apr_drg_description", "Code_illness", "Mortality risk", "Surg_Description", "Weight_baby", "Emergency dept_yes/No", "Tot_charg", "Tot_cost", "Result", "Payment Typology"]

## Feture Engineering 3RD

In [ ]:
imp_columns2 = ["Area_Service", "Hospital_Id", "Age", "Gender", "Days_spend_hsptl", "Admission_type", "Home or self care,", "ccs_diagnosis_code" "ccs_procedure_code", "apr_drg_description", "Code_illness","Weight_baby", "Mortality risk", "Surg_Description", "Emergency dept_yes/No", "Tot_charg", "Tot_cost", "Result"]

In [ ]:
len(imp_columns2)

In [ ]:
df = insurance_data[['Area_Service', 'Age', 'Gender', 'Cultural_group', 'ethnicity', 'Days_spend_hsptl',
        'Admission_type', 'Home or self care,', 'ccs_diagnosis_code',
        'ccs_procedure_code', 'Code_illness', "apr_drg_description", "Hospital County",
       'Mortality risk', 'Surg_Description', 'Emergency dept_yes/No',
       'Tot_charg', 'Tot_cost', 'Payment_Typology', "Result"]]

In [ ]:
df.shape

In [ ]:
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

In [ ]:
sns.countplot(df["Result"])

## Random Over Sampling

In [ ]:
X = df.drop(["Result"], axis = 1)
y = df["Result"]

In [ ]:
from imblearn.over_sampling import RandomOverSampler
random_over_sampling =  RandomOverSampler(sampling_strategy='minority')
X_ros, y_ros = random_over_sampling.fit_resample(X, y)

In [ ]:
from collections import Counter

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

In [ ]:
from sklearn.model_selection import train_test_split

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_ros, y_ros, test_size = 0.2)

In [ ]:
Xr_train.shape

In [ ]:
# applying decision tree classification
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree

dt = DecisionTreeClassifier(criterion='entropy',random_state=50)
dt.fit(Xr_train, yr_train)

In [ ]:
dt_pred = dt.predict(Xr_test)
confusion_matrix(dt_pred, yr_test)

In [ ]:
accuracy_score(dt_pred, yr_test)

## Outliers Handling

In [ ]:
# This function will handle outliers by imputing new value(median) or use capping
def handle_outliers(data, method):
    new_values = []
    
    Q1, Q3 = data.quantile([0.25, 0.75])
    IQR = Q3 - Q1

    lower_limit = Q1 - (1.5 * IQR)
    upper_limit = Q3 + (1.5 * IQR)
    
    if method == "imputation":
        median = data.median()
        
        for element in data:
            
            if (element < lower_limit) or (element > upper_limit):
                new_values.append(median)
                
            else:
                new_values.append(element)
    
    elif method == "capping":
        q1, q3 = data.quantile([0.25, 0.75])
        
        for element in data:
            
            if (element < lower_limit):
                new_values.append(q1)
                
            elif (element > upper_limit):
                new_values.append(q3)
                
            else:
                new_values.append(element)
                
    return new_values

In [ ]:
df["Tot_cost"] = handle_outliers(insurance_data["Tot_cost"], "capping")
df["Tot_charg"] = handle_outliers(insurance_data["Tot_charg"], "capping")

In [ ]:
plt.boxplot(df["Tot_cost"])

In [ ]:
plt.boxplot(df["Tot_charg"])

In [ ]:
genuine = df[df["Result"] == 1]
fraud = df[df["Result"] == 0]

## Random Under Sampling

In [ ]:
genuine = genuine.sample(fraud.shape[0])
genuine.shape

In [ ]:
new_df = pd.concat([genuine, fraud], axis = 0)

In [ ]:
new_df.head()

In [ ]:
new_df.shape

In [ ]:
sns.countplot(new_df["Result"])

In [ ]:
X = new_df.drop(["Result"], axis = 1)
y = new_df["Result"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 20)

## Running Algorithms

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
dt_clf_pred = dt_clf.predict(X_test)

confusion_matrix(dt_clf_pred, y_test)

In [ ]:
accuracy = accuracy_score(dt_clf_pred, y_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}")

In [ ]:
accuracy_train = accuracy_score(y_train, dt_clf.predict(X_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.shape

In [ ]:
ins_dt = df.copy()

In [ ]:
pd.DataFrame.drop_duplicates(ins_dt, inplace = True)
ins_dt.shape

## Random Over Sampling

In [ ]:
X = ins_dt.drop(["Result"], axis = 1)
y = ins_dt["Result"]

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
X_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

In [ ]:
X_ros_train, X_ros_test, y_ros_train, y_ros_test = train_test_split(X_ros, y_ros, test_size = .20)

In [ ]:
X_ros_train.shape

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gradient_clf = GradientBoostingClassifier(
    loss = 'exponential', 
    criterion = 'mse',
    n_estimators = 200,
    learning_rate=0.2,

)

In [ ]:
gradient_clf.fit(X_ros_train, y_ros_train)

In [ ]:
gd_pred = gradient_clf.predict(X_ros_test)
train_pred = gradient_clf.predict(X_ros_train)

In [ ]:
print("Train Accuracy: ")
accuracy_score(y_ros_train, train_pred)

In [ ]:
print("Test Accuracy: ")
accuracy_score(y_ros_test, gd_pred)

In [ ]:
X_ros_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(
    criterion='gini',
)

In [ ]:
dt_clf.fit(X_ros_train, y_ros_train)

In [ ]:
dt_clf_pred = dt_clf.predict(X_ros_test)

confusion_matrix(dt_clf_pred, y_ros_test)

In [ ]:
accuracy = accuracy_score(dt_clf_pred, y_ros_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}")

In [ ]:
accuracy_train = accuracy_score(y_ros_train, dt_clf.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators = 300,       ## Total Number of decision tree that will be used to train an ensemble is 2
    max_samples = 100,         ## each trained on 100 training instances randomly sampled from the training set with replacement
    #bootstrap = True,         ## Bootstrap = True means use bagging method, if this option is set to False then it will be Pasting method that we didn't mention here.
    n_jobs = -1               ## n_jobs means how many cores will be used to train the ensemble and -1 here means all of them
)

bag_clf.fit(X_ros_train, y_ros_train)

In [ ]:
pred = bag_clf.predict(X_ros_test)
confusion_matrix(y_ros_test, pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'gini', n_jobs = -1)

In [ ]:
forest_classifier.fit(X_ros_train, y_ros_train)
y_ros_pred = forest_classifier.predict(X_ros_test)

In [ ]:
confusion_matrix(y_ros_pred, y_ros_test)

In [ ]:
print(f"Accuracy: {round((accuracy_score(y_ros_pred, y_ros_test)*100), 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_ros_train, dt_clf.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# activation : {'identity', 'logistic', 'tanh', 'relu'}
# solver : {'lbfgs', 'sgd', 'adam'}
# learning_rate : {'constant', 'invscaling', 'adaptive'}

clf = MLPClassifier(hidden_layer_sizes = (40,), activation = 'relu', solver = "lbfgs")

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = [{'activation': ['identity', 'logistic', 'tanh', 'relu'],
          'solver' : ['lbfgs', 'sgd', 'adam'],
          'learning_rate' : ['constant', 'invscaling', 'adaptive']}]

grid_search = GridSearchCV(estimator = clf, param_grid = params, scoring = 'accuracy', cv = 10, n_jobs = -1)

In [ ]:
grid_search.fit(X_ros_train, y_ros_train)

In [ ]:
help(MLPClassifier)

In [ ]:
clf.fit(X_ros_train, y_ros_train)

In [ ]:
clf.predict_proba(X_test[:1])

In [ ]:
mlp_pred = clf.predict(X_ros_test)

In [ ]:
accuracy_score(mlp_pred, y_ros_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(criterion = "entropy", random_state = 50)

dt_clf.fit(X_ros_train, y_ros_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
dt_clf_pred = dt_clf.predict(X_ros_test)

confusion_matrix(dt_clf_pred, y_ros_test)

In [ ]:
accuracy = accuracy_score(dt_clf_pred, y_ros_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}")

In [ ]:
accuracy_train = accuracy_score(y_ros_train, dt_clf.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'gini', n_jobs = -1)

In [ ]:
forest_classifier.fit(X_ros_train, y_ros_train)
y_ros_pred = forest_classifier.predict(X_ros_test)
confusion_matrix(y_ros_pred, y_ros_test)

In [ ]:
accuracy = accuracy_score(y_ros_pred, y_ros_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_ros_train, dt_clf.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'gini', n_jobs = -1)

In [ ]:
forest_classifier.fit(X_df_train, y_df_train)
y_df_pred = forest_classifier.predict(X_df_test)
confusion_matrix(y_df_pred, y_df_test)

In [ ]:
accuracy = accuracy_score(y_df_pred, y_df_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_df_train, dt_clf.predict(X_df_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

In [ ]:
df = insurance_data[["Hospital County", "Hospital Id", "Cultural_group", "ethnicity", "Days_spend_hsptl", "Admission_type", "Home or self care,", "ccs_procedure_code", "apr_drg_description", "Code_illness", "Mortality risk", "Surg_Description", "Weight_baby", "Emergency dept_yes/No", "Tot_charg", "Tot_cost", "Payment_Typology", "Result"]]

In [ ]:
df.head()

In [ ]:
insurance_data.info()

In [ ]:
df.info()

In [ ]:
genuine = df[df["Result"] == 1]
fraud = df[df["Result"] == 0]

In [ ]:
genuine = genuine.sample(500000)

In [ ]:
new_df = pd.concat([genuine, fraud], axis = 0)

In [ ]:
new_df.head()

In [ ]:
new_df.info()

In [ ]:
sns.countplot(new_df["Result"])

In [ ]:
X = new_df.drop(columns = ["Result"])
y = new_df["Result"]

In [ ]:
from collections import Counter

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
X_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

In [ ]:
sns.countplot(y_ros)

In [ ]:
X_ros.shape

In [ ]:
X_ros_train, X_ros_test, y_ros_train, y_ros_test = train_test_split(X_ros, y_ros, test_size = .25)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_ros_train, y_ros_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
dt_clf_pred = dt_clf.predict(X_ros_test)

confusion_matrix(dt_clf_pred, y_ros_test)

In [ ]:
accuracy = accuracy_score(dt_clf_pred, y_ros_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}")

In [ ]:
accuracy_train = accuracy_score(y_ros_train, dt_clf.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

In [ ]:
sns.countplot(df["Result"])

In [ ]:
X_df = df.drop(["Result"], axis = 1)
y_df = df["Result"]
# X_df.head()

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler

ros2 = RandomOverSampler(random_state=42)

# fit predictor and target variable
X, y = ros2.fit_resample(X_df, y_df)

print('Original dataset shape', Counter(y_df))
print('Resample dataset shape', Counter(y))

In [ ]:
X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X, y, test_size = .30, random_state = 20)

### Random Over Sampling with decision tree

In [ ]:
dt_clf.fit(X_df_train, y_df_train)

In [ ]:
y_df_pred = dt_clf.predict(X_df_test)
confusion_matrix(y_df_pred, y_df_test)

In [ ]:
accuracy = accuracy_score(y_df_pred, y_df_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_df_train, dt_clf.predict(X_df_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

In [ ]:
dt_clf = DecisionTreeClassifier(criterion = 'gini', max_depth = 150, random_state = 30)

In [ ]:
dt_clf.fit(X_df_train, y_df_train)

In [ ]:
y_df_pred = dt_clf.predict(X_df_test)
confusion_matrix(y_df_pred, y_df_test)

In [ ]:
accuracy = accuracy_score(y_df_pred, y_df_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_df_train, dt_clf.predict(X_df_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

### Random Over Sampling with Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'gini', n_jobs = -1)

In [ ]:
forest_classifier.fit(X_df_train, y_df_train)
y_df_pred = forest_classifier.predict(X_df_test)
confusion_matrix(y_df_pred, y_df_test)

In [ ]:
accuracy = accuracy_score(y_df_pred, y_df_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_df_train, dt_clf.predict(X_df_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

### RandomForestClassifier with RandomOverSampling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_classifier = RandomForestClassifier(n_estimators = 300, criterion = 'gini', n_jobs = -1)

In [ ]:
forest_classifier.fit(X_df_train, y_df_train)
y_df_pred = forest_classifier.predict(X_df_test)
confusion_matrix(y_df_pred, y_df_test)

In [ ]:
accuracy = accuracy_score(y_df_pred, y_df_test)
acc = accuracy*100
print(f"Test Accuracy = {round(acc, 2)}%")

In [ ]:
accuracy_train = accuracy_score(y_df_train, forest_classifier.predict(X_df_train))
print(f"Train Accuracy: {round(accuracy_train*100, 2)}%")

### Random Forest With 200 estimators and gini

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# forest_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'gini', n_jobs = -1)
# fc_model = forest_classifier.fit(X_ros_train, y_ros_train)
# fc_ros_pred = fc_model.predict(X_ros_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# confusion_matrix(fc_ros_pred, y_ros_test)

In [ ]:
# accuracy = accuracy_score(fc_ros_pred, y_ros_test)
# acc = accuracy*100
# print(f"Test Accuracy = {round(acc, 2)}")

In [ ]:
# accuracy_train = accuracy_score(y_ros_train, fc_model.predict(X_ros_train))
# print(f"Train Accuracy: {round(accuracy_train*100, 2)}")

### Random Forest with 300 estimators and gini

In [ ]:
forest_classifier2 = RandomForestClassifier(n_estimators = 300, criterion = 'gini', n_jobs = -1)

In [ ]:
fc_model2 = forest_classifier2.fit(X_ros_train, y_ros_train)

In [ ]:
fc_ros_pred2 = fc_model2.predict(X_ros_test)

In [ ]:
# from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(fc_ros_pred2, y_ros_test)

In [ ]:
accuracy2 = accuracy_score(fc_ros_pred2, y_ros_test)
acc2 = accuracy2*100
print(f"Test Accuracy = {round(acc2, 2)}")

In [ ]:
accuracy_train2 = accuracy_score(y_ros_train, fc_model2.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train2*100, 2)}")

### Random Forest with 200 estimators and entropy

In [ ]:
forest_classifier3 = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', n_jobs = -1)

In [ ]:
fc_model3 = forest_classifier3.fit(X_ros_train, y_ros_train)

In [ ]:
fc_ros_pre3 = fc_model3.predict(X_ros_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(fc_ros_pred3, y_ros_test)

In [ ]:
accuracy3 = accuracy_score(fc_ros_pred3, y_ros_test)
acc3 = accuracy3*100
print(f"Test Accuracy = {round(acc3, 2)}")

In [ ]:
accuracy_train3 = accuracy_score(y_ros_train, fc_model3.predict(X_ros_train))
print(f"Train Accuracy: {round(accuracy_train3*100, 2)}")

In [ ]:
insurance_data.info()

## All the values which are greater than .1.

### We will take these columns for now.

In [ ]:
fe_insurance_data = insurance_data[["Area_Service", "Age", "Gender", "Cultural_group", "ethnicity", "Days_spend_hsptl", "Admission_type", "Tot_charg", "Tot_cost", "Result"]]

fe_insurance_data.head()

In [ ]:
fe_insurance_data.head()

In [ ]:
sns.countplot(fe_insurance_data["Result"])

In [ ]:
X = fe_insurance_data.drop(columns = ["Result"])
y = fe_insurance_data["Result"]

In [ ]:
# import library
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
plt.figure(figsize = (4, 5))
sns.countplot(y_smote)

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix

xs_train, xs_test, ys_train, ys_test = train_test_split(x_smote, y_smote, test_size = .30, random_state = 20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_validate

rf_model = RandomForestClassifier(n_estimators = 100, criterion = 'gini', n_jobs = -1, random_state = 50)

In [ ]:
rf_model.fit(xs_train, ys_train)

In [ ]:
score = cross_validate(rf_model, xs_train, ys_train)

In [ ]:
score

In [ ]:
accuracy_score(ys_train, rf_model.predict(xs_train))

In [ ]:
rf_pred = rf_model.predict(xs_test)

acc = accuracy_score(ys_test, rf_pred)

print(f"Test Accuracy : {round(acc*100, 2)}%")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def dec_tree(criter, max_depth):
    _accuracy = []
    for md in range(2, max_depth+1, 2):
        dt_clf = DecisionTreeClassifier(criterion = criter, max_depth = md, random_state=50)
        model = dt_clf.fit(xs_train, ys_train)
        predictions = model.predict(xs_test)
        acc = accuracy_score(predictions, ys_test)
        _accuracy.append(acc)
    return _accuracy

In [ ]:
gini_tree = dec_tree("gini", 101)
entr_tree = dec_tree("entropy", 101)

In [ ]:
acc_df = pd.DataFrame({"max_depth": range(2, 101, 2), "gini_accuracy": gini_tree, "entr_accuracy": entr_tree})

In [ ]:
acc_df

In [ ]:
from collections import Counter

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target variable
x_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

In [ ]:
xos_train, xos_test, yos_train, yos_test = train_test_split(x_ros, y_ros, test_size = 0.3, random_state = 50)

In [ ]:
rf_model.fit(xos_train, yos_train)

In [ ]:
score = cross_validate(rf_model, xos_train, yos_train)

In [ ]:
score

In [ ]:
accuracy_score(yos_train, rf_model.predict(xos_train))

In [ ]:
rf_pred2 = rf_model.predict(xos_test)

acc2 = accuracy_score(yos_test, rf_pred2)

print(f"Test Accuracy : {round(acc2*100, 2)}%")

In [ ]:
random_forest_model = RandomForestClassifier(n_estimators = 300, criterion = 'gini', n_jobs = -1, random_state = 50)

In [ ]:
random_forest_model.fit(xos_train, yos_train)

In [ ]:
score = cross_validate(random_forest_model, xos_train, yos_train)

In [ ]:
score

In [ ]:
accuracy_score(yos_train, random_forest_model.predict(xos_train))

In [ ]:
rf_pred2 = random_forest_model.predict(xos_test)

acc2 = accuracy_score(yos_test, rf_pred2)

print(f"Test Accuracy : {round(acc2*100, 2)}%")